In [ ]:
import pandas as pd
df_daily_US_county_heat_insights=pd.read_csv('E:\\Google_Heat\\Dataset_to_share\\Copy of daily_US_county_heat_insights.csv', sep=",")
df0 = pd.read_csv("E:\\Google_Heat\\uscounties.csv", sep=",")
#df0=df0[~df0['County Name'].str.contains('King')]
#df0['County Name'] = df0['County Name']+' County'
df_daily_US_county_heat_insights

df0.rename(columns={'county_full': 'sub_region_2'}, inplace=True)
df0.rename(columns={'state_name': 'sub_region_1'}, inplace=True)
df0

merged_df = df_daily_US_county_heat_insights.merge(df0[['sub_region_2','sub_region_1', 'lat', 'lng']], on=['sub_region_2','sub_region_1'], how='left')

unique_dates = merged_df['date'].unique()

df_list = []

# Create dataframes for each unique date
for date in unique_dates:
    df = merged_df[merged_df['date'] == date].copy()
    df_list.append(df)
import pandas as pd



In [ ]:
merged_df

In [ ]:
df_daily_US_county_heat_insights['date'] = pd.to_datetime(df_daily_US_county_heat_insights['date'])
df_daily_US_county_heat_insights[df_daily_US_county_heat_insights['sub_region_2_code']==6037.0]['date'][191]

In [ ]:
import pandas as pd

# Assuming you have a DataFrame called 'df' with a column 'state_column' containing state names

# Mapping dictionary for state abbreviations
state_abbreviations = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
    'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA',
    'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA',
    'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO',
    'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
    'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH',
    'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
    'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT',
    'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}

# Convert state names to abbreviations
df_daily_US_county_heat_insights['sub_region_1'] = df_daily_US_county_heat_insights['sub_region_1'].map(state_abbreviations)

# Print the updated DataFrame
df_daily_US_county_heat_insights


In [ ]:
df_daily_US_county_heat_insights['sub_region_2']=df_daily_US_county_heat_insights['sub_region_1']+': '+df_daily_US_county_heat_insights['sub_region_2'].str.replace(' County', '')

In [ ]:
import pandas as pd


new_df = pd.DataFrame()

# Iterate over each DataFrame in df_list
for i, df in enumerate(df_list[0:5]):
    # Extract the "heat_all" column from the current DataFrame
    print(len(df))
    #column_name = f"Column_{i+1}"
    #new_df[column_name] = list(df["heat_all"].values)

#new_df


In [ ]:
df_daily_US_county_heat_insights['sub_region_2_code']==1001.0

In [ ]:
# Assuming you have a list of dataframes called df_list

max_values = []
max_lat = []
max_lng = []
max_date=[]
for df in df_list:
    max_value = df['heat_all'].max()  # Maximum value of 'heat_all' column
    max_index = df['heat_all'].idxmax()  # Index of the maximum value
    
    # Retrieve corresponding values from 'lat' and 'lng' columns
    lat_value = df.loc[max_index, 'lat']
    lng_value = df.loc[max_index, 'lng']
    date_value = df.loc[max_index, 'date']
    max_values.append(max_value)
    max_lat.append(lat_value)
    max_lng.append(lng_value)
    max_date.append(date_value)

# Create a new dataframe with the maximum values and corresponding lat/lng values
result_df = pd.DataFrame({'Max_Date': max_date,'Max_Heat_All': max_values, 'Max_Lat': max_lat, 'Max_Lng': max_lng})

# Save the dataframe as a CSV file
result_df.to_csv('max_values.csv', index=False)

import matplotlib.pyplot as plt

# Assuming you have the dataframe result_df

# Plotting the values in a 2D scatter plot
plt.scatter(result_df['Max_Lng'], result_df['Max_Lat'], c=result_df['Max_Heat_All'], cmap='jet')
plt.colorbar(label='Max Heat All')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Maximum Heat All Values')

# Display the plot
plt.show()


import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import levy

# Remove rows with missing or non-finite values
result_df_cleaned = result_df.dropna(subset=['Max_Lat', 'Max_Lng'])

# Calculate the step lengths
latitudes = result_df_cleaned['Max_Lat'].astype(float)
longitudes = result_df_cleaned['Max_Lng'].astype(float)
step_lengths = np.sqrt((latitudes.diff()**2) + (longitudes.diff()**2))

# Visualize the step length distribution
plt.hist(step_lengths, bins='auto', density=True, alpha=0.7)
plt.xlabel('Step Length')
plt.ylabel('Probability Density')
plt.title('Step Length Distribution')
plt.show()

# Calculate summary statistics
mean_length = step_lengths.mean()
median_length = step_lengths.median()
std_length = step_lengths.std()

# Fit a Levy distribution
params = levy.fit(step_lengths[1:]+0.001)
best_fit = levy(*params)

# Perform a power-law analysis
x = np.sort(step_lengths[1:])
y = np.arange(1, len(x) + 1) / len(x)

plt.loglog(x, y, marker='.', linestyle='none')
plt.xlabel('Step Length')
plt.ylabel('Probability (log scale)')
plt.title('Power-law Analysis')
plt.show()

# Print the results
print("Summary Statistics:")
print(f"Mean: {mean_length}")
print(f"Median: {median_length}")
print(f"Standard Deviation: {std_length}")
print("\nLevy Distribution Parameters:")
print(f"Alpha: {params[0]}")
print(f"Location: {params[1]}")
step_lengths_heat_all=step_lengths

In [ ]:
max_values = []
max_lat = []
max_lng = []
max_date=[]
for df in df_list:
    max_value = df['heat_general_info'].max()  # Maximum value of 'heat_all' column
    max_index = df['heat_general_info'].idxmax()  # Index of the maximum value
    
    # Retrieve corresponding values from 'lat' and 'lng' columns
    lat_value = df.loc[max_index, 'lat']
    lng_value = df.loc[max_index, 'lng']
    date_value = df.loc[max_index, 'date']
    max_values.append(max_value)
    max_lat.append(lat_value)
    max_lng.append(lng_value)
    max_date.append(date_value)

# Create a new dataframe with the maximum values and corresponding lat/lng values
result_df = pd.DataFrame({'Max_Date': max_date,'Max_Heat_All': max_values, 'Max_Lat': max_lat, 'Max_Lng': max_lng})

# Save the dataframe as a CSV file
result_df.to_csv('max_values.csv', index=False)

import matplotlib.pyplot as plt

# Assuming you have the dataframe result_df

# Plotting the values in a 2D scatter plot
plt.scatter(result_df['Max_Lng'], result_df['Max_Lat'], c=result_df['Max_Heat_All'], cmap='jet')
plt.colorbar(label='Max Heat All')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Maximum Heat All Values')

# Display the plot
plt.show()


import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import levy

# Remove rows with missing or non-finite values
result_df_cleaned = result_df.dropna(subset=['Max_Lat', 'Max_Lng'])

# Calculate the step lengths
latitudes = result_df_cleaned['Max_Lat']
longitudes = result_df_cleaned['Max_Lng']
step_lengths = np.sqrt((latitudes.diff()**2) + (longitudes.diff()**2))

# Visualize the step length distribution
plt.hist(step_lengths, bins='auto', density=True, alpha=0.7)
plt.xlabel('Step Length')
plt.ylabel('Probability Density')
plt.title('Step Length Distribution')
plt.show()

# Calculate summary statistics
mean_length = step_lengths.mean()
median_length = step_lengths.median()
std_length = step_lengths.std()

# Fit a Levy distribution
params = levy.fit(step_lengths[1:])
best_fit = levy(*params)

# Perform a power-law analysis
x = np.sort(step_lengths[1:])
y = np.arange(1, len(x) + 1) / len(x)

plt.loglog(x, y, marker='.', linestyle='none')
plt.xlabel('Step Length')
plt.ylabel('Probability (log scale)')
plt.title('Power-law Analysis')
plt.show()

# Print the results
print("Summary Statistics:")
print(f"Mean: {mean_length}")
print(f"Median: {median_length}")
print(f"Standard Deviation: {std_length}")
print("\nLevy Distribution Parameters:")
print(f"Alpha: {params[0]}")
print(f"Location: {params[1]}")
step_lengths_heat_general_info=step_lengths

In [ ]:
df

In [ ]:
max_values = []
max_lat = []
max_lng = []
max_date=[]
for df in df_list:
    max_value = df['heat_illness'].max()  # Maximum value of 'heat_all' column
    max_index = df['heat_illness'].idxmax()  # Index of the maximum value
    
    # Retrieve corresponding values from 'lat' and 'lng' columns
    lat_value = df.loc[max_index, 'lat']
    lng_value = df.loc[max_index, 'lng']
    date_value = df.loc[max_index, 'date']
    max_values.append(max_value)
    max_lat.append(lat_value)
    max_lng.append(lng_value)
    max_date.append(date_value)

# Create a new dataframe with the maximum values and corresponding lat/lng values
result_df = pd.DataFrame({'Max_Date': max_date,'Max_Heat_All': max_values, 'Max_Lat': max_lat, 'Max_Lng': max_lng})

# Save the dataframe as a CSV file
result_df.to_csv('max_values.csv', index=False)

import matplotlib.pyplot as plt

# Assuming you have the dataframe result_df

# Plotting the values in a 2D scatter plot
plt.scatter(result_df['Max_Lng'], result_df['Max_Lat'], c=result_df['Max_Heat_All'], cmap='jet')
plt.colorbar(label='Max Heat All')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Maximum Heat All Values')

# Display the plot
plt.show()


import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import levy

# Remove rows with missing or non-finite values
result_df_cleaned = result_df.dropna(subset=['Max_Lat', 'Max_Lng'])

# Calculate the step lengths
latitudes = result_df_cleaned['Max_Lat']
longitudes = result_df_cleaned['Max_Lng']
step_lengths = np.sqrt((latitudes.diff()**2) + (longitudes.diff()**2))

# Visualize the step length distribution
plt.hist(step_lengths, bins='auto', density=True, alpha=0.7)
plt.xlabel('Step Length')
plt.ylabel('Probability Density')
plt.title('Step Length Distribution')
plt.show()

# Calculate summary statistics
mean_length = step_lengths.mean()
median_length = step_lengths.median()
std_length = step_lengths.std()

# Fit a Levy distribution
params = levy.fit(step_lengths[1:])
best_fit = levy(*params)

# Perform a power-law analysis
x = np.sort(step_lengths[1:])
y = np.arange(1, len(x) + 1) / len(x)

plt.loglog(x, y, marker='.', linestyle='none')
plt.xlabel('Step Length')
plt.ylabel('Probability (log scale)')
plt.title('Power-law Analysis')
plt.show()

# Print the results
print("Summary Statistics:")
print(f"Mean: {mean_length}")
print(f"Median: {median_length}")
print(f"Standard Deviation: {std_length}")
print("\nLevy Distribution Parameters:")
print(f"Alpha: {params[0]}")
print(f"Location: {params[1]}")
step_lengths_heat_illness=step_lengths

In [ ]:
max_values = []
max_lat = []
max_lng = []
max_date=[]
for df in df_list:
    max_value = df['heat_stroke_and_exhaustion'].max()  # Maximum value of 'heat_all' column
    max_index = df['heat_stroke_and_exhaustion'].idxmax()  # Index of the maximum value
    
    # Retrieve corresponding values from 'lat' and 'lng' columns
    lat_value = df.loc[max_index, 'lat']
    lng_value = df.loc[max_index, 'lng']
    date_value = df.loc[max_index, 'date']
    max_values.append(max_value)
    max_lat.append(lat_value)
    max_lng.append(lng_value)
    max_date.append(date_value)

# Create a new dataframe with the maximum values and corresponding lat/lng values
result_df = pd.DataFrame({'Max_Date': max_date,'Max_Heat_All': max_values, 'Max_Lat': max_lat, 'Max_Lng': max_lng})

# Save the dataframe as a CSV file
result_df.to_csv('max_values.csv', index=False)

import matplotlib.pyplot as plt

# Assuming you have the dataframe result_df

# Plotting the values in a 2D scatter plot
plt.scatter(result_df['Max_Lng'], result_df['Max_Lat'], c=result_df['Max_Heat_All'], cmap='jet')
plt.colorbar(label='Max Heat All')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Maximum Heat All Values')

# Display the plot
plt.show()


import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import levy

# Remove rows with missing or non-finite values
result_df_cleaned = result_df.dropna(subset=['Max_Lat', 'Max_Lng'])

# Calculate the step lengths
latitudes = result_df_cleaned['Max_Lat']
longitudes = result_df_cleaned['Max_Lng']
step_lengths = np.sqrt((latitudes.diff()**2) + (longitudes.diff()**2))

# Visualize the step length distribution
plt.hist(step_lengths, bins='auto', density=True, alpha=0.7)
plt.xlabel('Step Length')
plt.ylabel('Probability Density')
plt.title('Step Length Distribution')
plt.show()

# Calculate summary statistics
mean_length = step_lengths.mean()
median_length = step_lengths.median()
std_length = step_lengths.std()

# Fit a Levy distribution
params = levy.fit(step_lengths[1:])
best_fit = levy(*params)

# Perform a power-law analysis
x = np.sort(step_lengths[1:])
y = np.arange(1, len(x) + 1) / len(x)

plt.loglog(x, y, marker='.', linestyle='none')
plt.xlabel('Step Length')
plt.ylabel('Probability (log scale)')
plt.title('Power-law Analysis')
plt.show()

# Print the results
print("Summary Statistics:")
print(f"Mean: {mean_length}")
print(f"Median: {median_length}")
print(f"Standard Deviation: {std_length}")
print("\nLevy Distribution Parameters:")
print(f"Alpha: {params[0]}")
print(f"Location: {params[1]}")
step_lengths_heat_stroke_and_exhaustion=step_lengths

In [ ]:
max_values = []
max_lat = []
max_lng = []
max_date=[]
for df in df_list:
    max_value = df['heat_air_conditioning'].max()  # Maximum value of 'heat_all' column
    max_index = df['heat_air_conditioning'].idxmax()  # Index of the maximum value
    
    # Retrieve corresponding values from 'lat' and 'lng' columns
    lat_value = df.loc[max_index, 'lat']
    lng_value = df.loc[max_index, 'lng']
    date_value = df.loc[max_index, 'date']
    max_values.append(max_value)
    max_lat.append(lat_value)
    max_lng.append(lng_value)
    max_date.append(date_value)

# Create a new dataframe with the maximum values and corresponding lat/lng values
result_df = pd.DataFrame({'Max_Date': max_date,'Max_Heat_All': max_values, 'Max_Lat': max_lat, 'Max_Lng': max_lng})

# Save the dataframe as a CSV file
result_df.to_csv('max_values.csv', index=False)

import matplotlib.pyplot as plt

# Assuming you have the dataframe result_df

# Plotting the values in a 2D scatter plot
plt.scatter(result_df['Max_Lng'], result_df['Max_Lat'], c=result_df['Max_Heat_All'], cmap='jet')
plt.colorbar(label='Max Heat All')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Maximum Heat All Values')

# Display the plot
plt.show()


import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import levy

# Remove rows with missing or non-finite values
result_df_cleaned = result_df.dropna(subset=['Max_Lat', 'Max_Lng'])

# Calculate the step lengths
latitudes = result_df_cleaned['Max_Lat']
longitudes = result_df_cleaned['Max_Lng']
step_lengths = np.sqrt((latitudes.diff()**2) + (longitudes.diff()**2))
step_lengths_heat_air_conditioning = step_lengths
# Visualize the step length distribution
plt.hist(step_lengths, bins='auto', density=True, alpha=0.7)
plt.xlabel('Step Length')
plt.ylabel('Probability Density')
plt.title('Step Length Distribution')
plt.show()

# Calculate summary statistics
mean_length = step_lengths.mean()
median_length = step_lengths.median()
std_length = step_lengths.std()

# Fit a Levy distribution
params = levy.fit(step_lengths[1:])
best_fit = levy(*params)

# Perform a power-law analysis
x = np.sort(step_lengths[1:])
y = np.arange(1, len(x) + 1) / len(x)

plt.loglog(x, y, marker='.', linestyle='none')
plt.xlabel('Step Length')
plt.ylabel('Probability (log scale)')
plt.title('Power-law Analysis')
plt.show()

# Print the results
print("Summary Statistics:")
print(f"Mean: {mean_length}")
print(f"Median: {median_length}")
print(f"Standard Deviation: {std_length}")
print("\nLevy Distribution Parameters:")
print(f"Alpha: {params[0]}")
print(f"Location: {params[1]}")
fig = plt.figure(figsize=(18, 6))

plt.plot(step_lengths[1:])
step_lengths_heat_air_conditioning=step_lengths

In [ ]:
import numpy as np
from scipy.stats import levy

# Assuming you have a dataset called 'data' that represents step lengths

# Fit the dataset to a Levy distribution using Maximum Likelihood Estimation (MLE)
params = levy.fit(step_lengths[1:])  # loc=0 assumes a starting point at the origin

# Extract the estimated shape parameter (alpha) and scale parameter (c)
shape_parameter = params[0]
scale_parameter = params[1]

# Print the estimated parameters
print("Estimated Parameters:")
print(f"Shape Parameter (alpha): {shape_parameter}")
print(f"Scale Parameter (c): {scale_parameter}")


In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import glob
import os

lframes = []
for idf in range(0, 730):
    mean_value = df_list[idf]['heat_general_info'].mean()
    filtered_df = df_list[idf][df_list[idf]['heat_general_info'] > mean_value]

    frame_data = go.Scatter(
        x=filtered_df['lng'],
        y=filtered_df['lat'],
        mode='markers',
        marker=dict(
            size=6,
            color=filtered_df['heat_general_info'],
            colorscale='Viridis',
            opacity=0.4,
            colorbar=dict(thickness=10, orientation='h')
        )
    )

    annotation = dict(
        text=f"Date: {unique_dates[idf]}",
        showarrow=False,
        xref="paper",
        yref="paper",
        x=0.05,
        y=0.05,
        bgcolor="white",
        opacity=0.8
    )

    lframes.append(go.Frame(data=[frame_data], name=f"Date: {unique_dates[idf]}", layout=go.Layout(annotations=[annotation])))

fig = go.Figure(
    data=[go.Scatter(
        x=filtered_df['lng'],
        y=filtered_df['lat'],
        mode='markers',
        marker=dict(
            symbol='square',
            size=6,
            color=filtered_df['heat_general_info'],
            colorscale='Viridis',
            opacity=0.4,
            colorbar=dict(thickness=10, orientation='h')
        )
    )],
    layout=go.Layout(
        xaxis=dict(range=[min(filtered_df['lng']), max(filtered_df['lat'])], autorange=True),
        yaxis=dict(range=[min(filtered_df['lng']), max(filtered_df['lat'])], autorange=True),
        sliders=[dict(
            active=0,
            currentvalue=dict(
                prefix="Frame: ",
                font=dict(color="#666"),
                xanchor="left",
                visible=True,
                offset=10
            ),
            pad=dict(t=50),
            len=0.9,
            x=0.1,
            y=0,
            steps=[dict(
                method="animate",
                args=[[f"Date: {unique_dates[idf]}"], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate"}],
                label=f"Date: {unique_dates[idf]}"
            ) for idf in range(365)]
        )],
        updatemenus=[
            dict(
                type="buttons",
                buttons=[
                    dict(
                        label="Play",
                        method="animate",
                        args=[None]
                    ),
                    dict(
                        label="Pause",
                        method="animate",
                        args=[[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate"}]
                    )
                ]
            )
        ]
    ),
    frames=lframes
)

fig.update_layout(
    width=1100,
    height=800
)
fig.write_html('us_heat_general_info.html')

fig.show()


In [ ]:
result_df

In [ ]:
result_df['Max_Date'] = pd.to_datetime(result_df['Max_Date'])
result_df

In [ ]:
fig = plt.figure(figsize=(18, 6))

plt.plot(step_lengths_heat_all)
plt.plot(step_lengths_heat_general_info)
plt.plot(step_lengths_heat_illness)
plt.plot(step_lengths_heat_stroke_and_exhaustion)
plt.plot(step_lengths_heat_air_conditioning)


array1 = np.array(list(step_lengths_heat_all[1:]))
array2 = np.array(list(step_lengths_heat_general_info[1:]))
array3 = np.array(list(step_lengths_heat_illness[1:]))
array4 = np.array(list(step_lengths_heat_stroke_and_exhaustion[1:]))
array5 = np.array(list(step_lengths_heat_air_conditioning[1:]))

print("Correlation step_lengths_heat_all, step_lengths_heat_general_info:  ")
correlation = np.corrcoef(array1, array2)[0, 1]
print(correlation)
correlation = np.corrcoef(array1, array3)[0, 1]
print("Correlation step_lengths_heat_all, step_lengths_heat_illness:  ")
print(correlation)
correlation = np.corrcoef(array1, array4)[0, 1]
print("Correlation step_lengths_heat_all, step_lengths_heat_stroke_and_exhaustion:  ")
print(correlation)
correlation = np.corrcoef(array1, array5)[0, 1]
print("Correlation step_lengths_heat_all, step_lengths_heat_air_conditioning:  ")
print(correlation)


correlation = np.corrcoef(array2, array3)[0, 1]
print("Correlation step_lengths_heat_general_info, step_lengths_heat_illness:  ")
print(correlation)
correlation = np.corrcoef(array2, array4)[0, 1]
print("Correlation step_lengths_heat_general_info, step_lengths_heat_stroke_and_exhaustion:  ")
print(correlation)
correlation = np.corrcoef(array2, array5)[0, 1]
print("Correlation step_lengths_heat_general_info, step_lengths_heat_air_conditioning:  ")
print(correlation)



In [ ]:


correlation

In [ ]:
list(step_lengths_heat_all)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Assuming you have the 'result_df' DataFrame and 'step_lengths' time series

# Convert 'Max_Date' to datetime format

# Set up the figure and axes
fig, ax = plt.subplots(figsize=(18, 6))

# Plot the scatter plot with different colors for each weekday
scatter = ax.scatter(result_df['Max_Date'][1:], step_lengths_heat_general_info[1:], c=result_df['Max_Date'][1:].dt.weekday, cmap='viridis')

# Set x-axis label and tick format
ax.set_xlabel('Date')
ax.xaxis.set_major_locator(mdates.WeekdayLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))

# Set y-axis label
ax.set_ylabel('Step Length')

# Add colorbar
cbar = plt.colorbar(scatter, ticks=range(7))
cbar.set_label('Weekday')
cbar.set_ticklabels(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])

# Rotate x-axis labels if needed
plt.xticks(rotation=45)

# Show the plot
plt.show()


In [ ]:

# simple code to implement Runs 
# test of randomnes
#Compare the value of the calculated Z-statistic with Zcritical  for a given level of confidence (Zcritical =1.96 for 
#confidence level of 95%) . he null hypothesis is rejected i.e. the numbers are declared not to be random, if |Z|>Zcritical.            
import random
import math
import statistics
  
  
def runsTest(l, l_median):
  
    runs, n1, n2 = 0, 0, 0
      
    # Checking for start of new run
    for i in range(len(l)):
          
        # no. of runs
        if (l[i] >= l_median and l[i-1] < l_median) or \
                (l[i] < l_median and l[i-1] >= l_median):
            runs += 1  
          
        # no. of positive values
        if(l[i]) >= l_median:
            n1 += 1   
          
        # no. of negative values
        else:
            n2 += 1   
  
    runs_exp = ((2*n1*n2)/(n1+n2))+1
    stan_dev = math.sqrt((2*n1*n2*(2*n1*n2-n1-n2))/ \
                       (((n1+n2)**2)*(n1+n2-1)))
  
    z = (runs-runs_exp)/stan_dev
  
    return z
    
l = list(step_lengths[1:])
      
l_median= statistics.median(l)
  
Z = abs(runsTest(l, l_median))
  
print('Z-statistic= ', Z)

In [ ]:
#!/usr/bin/env python3
# -*- coding: UTF-8 -*-
###                   https://github.com/Mottl/hurst/tree/master
"""
Hurst exponent and RS-analysis
https://en.wikipedia.org/wiki/Hurst_exponent
https://en.wikipedia.org/wiki/Rescaled_range
"""

name = "hurst"
__version__ = '0.0.5'

import sys
import math
import warnings
import numpy as np
try:
    import pandas as pd
except:
    pass

def __to_inc(x):
    incs = x[1:] - x[:-1]
    return incs

def __to_pct(x):
    pcts = x[1:] / x[:-1] - 1.
    return pcts

def __get_simplified_RS(series, kind):
    """
    Simplified version of rescaled range

    Parameters
    ----------

    series : array-like
        (Time-)series
    kind : str
        The kind of series (refer to compute_Hc docstring)
    """

    if kind == 'random_walk':
        incs = __to_inc(series)
        R = max(series) - min(series)  # range in absolute values
        S = np.std(incs, ddof=1)
    elif kind == 'price':
        pcts = __to_pct(series)
        R = max(series) / min(series) - 1. # range in percent
        S = np.std(pcts, ddof=1)
    elif kind == 'change':
        incs = series
        _series = np.hstack([[0.],np.cumsum(incs)])
        R = max(_series) - min(_series)  # range in absolute values
        S = np.std(incs, ddof=1)

    if R == 0 or S == 0:
        return 0  # return 0 to skip this interval due the undefined R/S ratio

    return R / S

def __get_RS(series, kind):
    """
    Get rescaled range (using the range of cumulative sum
    of deviations instead of the range of a series as in the simplified version
    of R/S) from a time-series of values.

    Parameters
    ----------

    series : array-like
        (Time-)series
    kind : str
        The kind of series (refer to compute_Hc docstring)
    """

    if kind == 'random_walk':
        incs = __to_inc(series)
        mean_inc = (series[-1] - series[0]) / len(incs)
        deviations = incs - mean_inc
        Z = np.cumsum(deviations)
        R = max(Z) - min(Z)
        S = np.std(incs, ddof=1)

    elif kind == 'price':
        incs = __to_pct(series)
        mean_inc = np.sum(incs) / len(incs)
        deviations = incs - mean_inc
        Z = np.cumsum(deviations)
        R = max(Z) - min(Z)
        S = np.std(incs, ddof=1)

    elif kind == 'change':
        incs = series
        mean_inc = np.sum(incs) / len(incs)
        deviations = incs - mean_inc
        Z = np.cumsum(deviations)
        R = max(Z) - min(Z)
        S = np.std(incs, ddof=1)

    if R == 0 or S == 0:
        return 0  # return 0 to skip this interval due undefined R/S

    return R / S

def compute_Hc(series, kind="random_walk", min_window=3, max_window=300, simplified=True):
    """
    Compute H (Hurst exponent) and C according to Hurst equation:
    E(R/S) = c * T^H

    Refer to:
    https://en.wikipedia.org/wiki/Hurst_exponent
    https://en.wikipedia.org/wiki/Rescaled_range
    https://en.wikipedia.org/wiki/Random_walk

    Parameters
    ----------

    series : array-like
        (Time-)series

    kind : str
        Kind of series
        possible values are 'random_walk', 'change' and 'price':
        - 'random_walk' means that a series is a random walk with random increments;
        - 'price' means that a series is a random walk with random multipliers;
        - 'change' means that a series consists of random increments
            (thus produced random walk is a cumulative sum of increments);

    min_window : int, default 10
        the minimal window size for R/S calculation

    max_window : int, default is the length of series minus 1
        the maximal window size for R/S calculation

    simplified : bool, default True
        whether to use the simplified or the original version of R/S calculation

    Returns tuple of
        H, c and data
        where H and c — parameters or Hurst equation
        and data is a list of 2 lists: time intervals and R/S-values for correspoding time interval
        for further plotting log(data[0]) on X and log(data[1]) on Y
    """

    if len(series)<100:
        raise ValueError("Series length must be greater or equal to 100")

    ndarray_likes = [np.ndarray]
    if "pandas.core.series" in sys.modules.keys():
        ndarray_likes.append(pd.core.series.Series)

    # convert series to numpy array if series is not numpy array or pandas Series
    if type(series) not in ndarray_likes:
        series = np.array(series)

    if "pandas.core.series" in sys.modules.keys() and type(series) == pd.core.series.Series:
        if series.isnull().values.any():
            raise ValueError("Series contains NaNs")
        series = series.values  # convert pandas Series to numpy array
    elif np.isnan(np.min(series)):
        raise ValueError("Series contains NaNs")

    if simplified:
        RS_func = __get_simplified_RS
    else:
        RS_func = __get_RS


    err = np.geterr()
    np.seterr(all='raise')

    max_window = max_window or len(series)-1
    window_sizes = list(map(
        lambda x: int(10**x),
        np.arange(math.log10(min_window), math.log10(max_window), 0.25)))
    window_sizes.append(len(series))

    RS = []
    for w in window_sizes:
        rs = []
        for start in range(0, len(series), w):
            if (start+w)>len(series):
                break
            _ = RS_func(series[start:start+w], kind)
            if _ != 0:
                rs.append(_)
        RS.append(np.mean(rs))

    A = np.vstack([np.log10(window_sizes), np.ones(len(RS))]).T
    H, c = np.linalg.lstsq(A, np.log10(RS), rcond=-1)[0]
    np.seterr(**err)

    c = 10**c
    return H, c, [window_sizes, RS]

def random_walk(length, proba=0.5, min_lookback=1, max_lookback=100, cumprod=False):
    """
    Generates a random walk series

    Parameters
    ----------

    proba : float, default 0.5
        the probability that the next increment will follow the trend.
        Set proba > 0.5 for the persistent random walk,
        set proba < 0.5 for the antipersistent one

    min_lookback: int, default 1
    max_lookback: int, default 100
        minimum and maximum window sizes to calculate trend direction
    cumprod : bool, default False
        generate a random walk as a cumulative product instead of cumulative sum
    """

    assert(min_lookback>=1)
    assert(max_lookback>=min_lookback)

    if max_lookback > length:
        max_lookback = length
        warnings.warn("max_lookback parameter has been set to the length of the random walk series.")

    if not cumprod:  # ordinary increments
        series = [0.] * length  # array of prices
        for i in range(1, length):
            if i < min_lookback + 1:
                direction = np.sign(np.random.randn())
            else:
                lookback = np.random.randint(min_lookback, min(i-1, max_lookback)+1)
                direction = np.sign(series[i-1] - series[i-1-lookback]) * np.sign(proba - np.random.uniform())
            series[i] = series[i-1] + np.fabs(np.random.randn()) * direction
    else:  # percent changes
        series = [1.] * length  # array of prices
        for i in range(1, length):
            if i < min_lookback + 1:
                direction = np.sign(np.random.randn())
            else:
                lookback = np.random.randint(min_lookback, min(i-1, max_lookback)+1)
                direction = np.sign(series[i-1] / series[i-1-lookback] - 1.) * np.sign(proba - np.random.uniform())
            series[i] = series[i-1] * np.fabs(1 + np.random.randn()/1000. * direction)

    return series




# Use random_walk() function or generate a random walk series manually:
# series = random_walk(99999, cumprod=True)
np.random.seed(42)
random_changes = 1. + np.random.randn(99999) / 1000.
series = np.cumprod(random_changes)  # create a random walk from random changes

# Evaluate Hurst equation
H, c, data = compute_Hc(np.array(step_lengths[1:]+0.01), kind='change', simplified=True)

# Plot
# uncomment the following to make a plot using Matplotlib:
"""
import matplotlib.pyplot as plt

f, ax = plt.subplots()
ax.plot(data[0], c*data[0]**H, color="deepskyblue")
ax.scatter(data[0], data[1], color="purple")
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('Time interval')
ax.set_ylabel('R/S ratio')
ax.grid(True)
plt.show()
"""

print("H={:.4f}, c={:.4f}".format(H,c))



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from hurst import compute_Hc, random_walk


series = np.array(step_lengths[1:]+0.01)  

H, c, data = compute_Hc(series, kind='change', simplified=True)

# Plot
f, ax = plt.subplots()
ax.plot(data[0], c*data[0]**H, color="deepskyblue")
ax.scatter(data[0], data[1], color="purple")
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('Time interval')
ax.set_ylabel('R/S ratio')
ax.grid(True)
plt.show()

print("H={:.4f}, c={:.4f}".format(H,c))

In [ ]:
!pip install pycwt 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pywt

# Assuming you have spatiotemporal data with two variables 'var1' and 'var2'

# Set the parameters for the wavelet transform
scales = np.arange(1, 10)
wavelet_name = 'morl'

# Perform the cross wavelet transform
cwt_var1, frequencies = pywt.cwt(array1, scales, wavelet_name)
cwt_var2, _ = pywt.cwt(array5, scales, wavelet_name)

# Plot the cross wavelet transform
plt.figure(figsize=(12, 8))
plt.imshow(np.abs(cwt_var1) * np.abs(cwt_var2), aspect='auto', cmap='jet')
plt.colorbar(label='Magnitude')
plt.xlabel('Time')
plt.ylabel('Scale')
plt.title('Cross Wavelet Transform')
plt.show()


In [ ]:
array1 = np.array(list(step_lengths_heat_all[1:]))
array2 = np.array(list(step_lengths_heat_general_info[1:]))
result_df_step_lengths_heat_all=result_df.copy()
result_df_step_lengths_heat_all['Max_Heat_All']=step_lengths_heat_all
result_df_step_lengths_heat_all.rename(columns={'Max_Heat_All': 'heat_all'}, inplace=True)
result_df_step_lengths_heat_general_info=result_df.copy()
result_df_step_lengths_heat_general_info['Max_Heat_All']=step_lengths_heat_general_info
result_df_step_lengths_heat_general_info.rename(columns={'Max_Heat_All': 'heat_general_info'}, inplace=True)


In [ ]:
result_df_step_lengths_heat_general_info

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# Assuming you have temperature and pressure DataFrames
result_df_step_lengths_heat_all

result_df_step_lengths_heat_general_info
# Merge temperature and pressure DataFrames based on common columns
merged_df12 = pd.merge(result_df_step_lengths_heat_all, result_df_step_lengths_heat_general_info, on=['Max_Date', 'Max_Lat', 'Max_Lng'])

merged_df12